In [3]:
#from nba_api.stats.static import players
#
#
# players.find_players_by_full_name("Lebron James")
#
#players.find_player_by_id(2544)

In [1]:
#Module load
from nba_api.stats.endpoints import commonallplayers #per avere l'elenco dei giocatori (id)
from nba_api.stats.endpoints import playercareerstats #per avere i dati
from nba_api.stats.endpoints import playerawards #per sapere chi ha vinto MVP
from nba_api.stats.endpoints import teamyearbyyearstats #classifiche finali delle squadre
import pandas as pd
from tqdm import tqdm
import pickle

In [7]:
from_year = 2003

total_players = commonallplayers.CommonAllPlayers().get_data_frames()[0]
total_players['TO_YEAR'] = total_players['TO_YEAR'].astype('int')
total_players = total_players[total_players['TO_YEAR'] > from_year]

#Ho controllato che in total_players, il PERSON_ID sia univoco
total_players

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
5,949,"Abdur-Rahim, Shareef",Shareef Abdur-Rahim,0,1996,2007,shareef_abdur-rahim,shareef_abdur-rahim,0,,,,,None,Y,00
9,203518,"Abrines, Alex",Alex Abrines,0,2016,2018,alex_abrines,alex_abrines,0,,,,,None,Y,00
10,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2021,precious_achiuwa,precious_achiuwa,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
11,101165,"Acker, Alex",Alex Acker,0,2005,2008,alex_acker,alex_acker,0,,,,,None,Y,01
15,203112,"Acy, Quincy",Quincy Acy,0,2012,2018,quincy_acy,quincy_acy,0,,,,,None,Y,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4717,2583,"Zimmerman, Derrick",Derrick Zimmerman,0,2005,2005,derrick_zimmerman,derrick_zimmerman,0,,,,,None,Y,01
4718,1627757,"Zimmerman, Stephen",Stephen Zimmerman,0,2016,2016,stephen_zimmerman,stephen_zimmerman,0,,,,,None,Y,01
4719,1627835,"Zipser, Paul",Paul Zipser,0,2016,2017,paul_zipser,paul_zipser,0,,,,,None,Y,01
4720,1627790,"Zizic, Ante",Ante Zizic,0,2017,2019,ante_zizic,ante_zizic,0,,,,,None,Y,01


In [40]:
# total_names = total_players[["PERSON_ID", "DISPLAY_FIRST_LAST"]]
# total_stats = pd.DataFrame()

In [41]:
# stats = playercareerstats.PlayerCareerStats(player_id=2544).get_data_frames()[0]
# stats = stats.drop(stats.columns[[2, 4, 7]], axis=1)
# awards = playerawards.PlayerAwards(player_id=2544).get_data_frames()[0]
# awards = awards.drop(awards.columns[[1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13]], axis=1)
# awards = awards.rename(columns={"SEASON": "SEASON_ID"})
# awards = awards.rename(columns={"PERSON_ID": "PLAYER_ID"})
# awards = awards.loc[awards["DESCRIPTION"]=="NBA Most Valuable Player"]

In [31]:
all_teams_history = list(total_players["TEAM_ID"].drop_duplicates())
data_all_team_stats = pd.DataFrame()

for team in all_teams_history:
    t = teamyearbyyearstats.TeamYearByYearStats(team_id=team).get_data_frames()[0]
    data_all_team_stats = pd.concat([data_all_team_stats, t], axis=0, ignore_index=True)

#data_all_team_stats = [int("20" + x.split("-")[1]) for x in all_teams_history] NON SERVE, FACCIO DOPO -- ELIMINARE

data_all_team_stats = data_all_team_stats.rename(columns={"YEAR": "SEASON_ID"})
data_all_team_stats = data_all_team_stats[["TEAM_ID", "SEASON_ID", "CONF_RANK"]]

with open("data_all_team_stats.pkl",'wb') as f:
    pickle.dump(data_all_team_stats,f)

data_all_team_stats


KeyboardInterrupt: 

,PERSON_ID,DISPLAY_FIRST_LAST
2297,1628994,George King
2301,1629663,Louis King
2308,200814,Tarence Kinsey
2309,1905,Andrei Kirilenko
2310,203945,Alex Kirk
...,...,...
4717,2583,Derrick Zimmerman
4718,1627757,Stephen Zimmerman
4719,1627835,Paul Zipser
4720,1627790,Ante Zizic


In [9]:
total_names = total_players[["PERSON_ID", "DISPLAY_FIRST_LAST"]]
total_stats = pd.DataFrame()
with open("data_all_team_stats.pkl", "rb") as f:
    data_all_team_stats = pickle.load(f)

for index, row in tqdm(total_names.iterrows()):
    stats = playercareerstats.PlayerCareerStats(player_id=row["PERSON_ID"]).get_data_frames()[0]

    stats = stats.drop(stats.columns[[2, 4, 7]], axis=1)

    awards = playerawards.PlayerAwards(player_id=row["PERSON_ID"]).get_data_frames()[0]

    awards = awards.drop(awards.columns[[0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13]], axis=1) #Tolgo anche PERSON_ID
    awards = awards.rename(columns={"SEASON": "SEASON_ID"})
    awards = awards.loc[awards["DESCRIPTION"]=="NBA Most Valuable Player"]

    #merge
    stats=stats.merge(awards, how="left", on="SEASON_ID")
    #1 to MVP, 0 to other
    stats["DESCRIPTION"] = stats["DESCRIPTION"].replace("NBA Most Valuable Player", 1) #Replace the description MVP with 1
    stats["DESCRIPTION"] = stats["DESCRIPTION"].fillna(0) #Replace NULL with 0
    stats = stats.rename(columns={"DESCRIPTION": "MVP"})

    #merge della classifica in cui è arrivata la squadra in una certa stagione
    stats = stats.merge(data_all_team_stats, how="inner", on=["SEASON_ID", "TEAM_ID"])

    total_stats = pd.concat([total_stats, stats],axis=0,ignore_index=True)


#Tolgo la formattazione "year1-year2" (2010-11) e lascio solo il secondo anno (2011)
total_stats["SEASON_ID"] = [int("20"+x.split("-")[1]) for x in list(total_stats["SEASON_ID"])] #Problema se si prendono dati < anno 2000
#Prendo i dati superiore all'anno FROM_YEAR
total_stats= total_stats[total_stats["SEASON_ID"] > from_year]


with open("player_stats.pkl",'wb') as f:
    pickle.dump(total_stats,f)


25it [01:15,  3.00s/it]


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [6]:
with open("player_stats.pkl", "rb") as f:
    total_stats = pickle.load(f)

total_stats


0       2017
1       2018
2       2019
3       2021
4       2022
        ... 
8920    2019
8921    2019
8922    2020
8923    2021
8924    2022
Name: SEASON_ID, Length: 6826, dtype: int64

In [42]:
pearson = total_stats.astype("int").corr()
#pd.concat(pearson[pearson["MVP"]>0.15], pearson[pearson["MVP"]> 0.15], axis=0)
pearson[(pearson["MVP"]>0.15) | (pearson["MVP"]<-0.15)]

,PLAYER_ID,SEASON_ID,TEAM_ID,PLAYER_AGE,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,MVP,CONF_RANK
FTM,-0.197669,0.043901,0.023010,0.009837,0.622324,0.799861,0.891722,0.886047,-0.058772,0.531831,...,0.728782,0.692090,0.724106,0.724981,0.475132,0.873958,0.685088,0.923268,0.179188,-0.038329
FTA,-0.202541,0.051645,0.025372,-0.005106,0.641486,0.811104,0.893580,0.878098,-0.060964,0.477803,...,0.777602,0.751284,0.704630,0.732259,0.540221,0.880450,0.719506,0.915767,0.172407,-0.033547
AST,-0.173566,0.038471,-0.001729,0.083638,0.594145,0.750309,0.752335,0.777350,-0.058767,0.583019,...,0.548232,0.483457,1.000000,0.792204,0.230881,0.877867,0.586659,0.768996,0.153777,-0.045185
TOV,-0.217895,0.053348,0.012568,0.022171,0.727773,0.879405,0.906873,0.913172,-0.072654,0.576077,...,0.766324,0.724113,0.877867,0.834372,0.478676,1.000000,0.785611,0.912914,0.151261,-0.013662
PTS,-0.201023,0.027046,0.014441,0.038940,0.755048,0.923840,0.992827,0.990290,-0.073256,0.698064,...,0.804850,0.759934,0.768996,0.812381,0.514629,0.912914,0.792156,1.000000,0.156608,-0.050980
MVP,-0.024397,-0.007285,-0.014525,-0.003921,0.050185,0.080931,0.145543,0.131506,-0.003045,0.109570,...,0.109425,0.094419,0.153777,0.110069,0.045331,0.151261,0.051452,0.156608,1.000000,-0.058519


#How it's structured this dataset
- Player_id --> id del giocatore(da ritrovare il nome con players.find_player_by_id())
- Season_id --> id della stagione
- ...other...
- GP --> Games Played
- GS --> Games ... (idk but similar to GP) (I think is not usefull)
- MIN Minutes Played
- FGM Field Goals Made --> Numero canestri fatti
- FGA Field Goals Attempted --> Numero tiri provati
- FG% Field Goal Percentage --> FGM/FGA, rapporto precisione tiri
- 3PM 3 Point Field Goals Made --> Numero canestri da 3 fatti
- 3PA 3 Point Field Goals Attempted --> Numero tiri da 3 provati
- 3P% 3 Point Field Goals Percentage --> 3PM/3PA, precisione tiri da 3
- FTM Free Throws Made --> Tiri liberi fatti
- FTA Free Throws Attempted --> tiri liberi provati
- FT% Free Throw Percentage --> FTM/FTA
- OREB Offensive Rebounds --> Rimbalzi conquistati in attacco
- DREB Defensive Rebounds --> rimbalzi conquistati in difesa
- REB Rebounds --> OREB + DREB, rimbalzi totali
- AST Assists
- TOV Turnovers --> palla persa
- STL Steals --> palla rubata
- BLK Blocks --> blocchi
- PF Personal Fouls --> falli personali
- DD2 Double Doubles --> doppie doppie NON C'È
- TD3 Trible Doubles --> triple doppie NON C'È
- PTS Points

In [53]:
#How is unbalanced
total_stats["MVP"].value_counts(normalize=True)


0.0    0.998389
1.0    0.001611
Name: MVP, dtype: float64

In [10]:
from sklearn.decomposition import PCA
#Fatto a caso, non finito

mvp = total_stats["MVP"]
id_player = total_stats[["PLAYER_ID", "SEASON_ID", "TEAM_ID", "PLAYER_AGE"]]

data_to_evaluate = total_stats.drop(["MVP", "PLAYER_ID", "SEASON_ID", "TEAM_ID", "PLAYER_AGE"], axis=1)

pca = PCA(n_components=7)
pca_data = pca.fit(data_to_evaluate)
print(pca_data.explained_variance_ratio_)
print(pca_data.singular_values_)


[9.33967771e-01 3.15349868e-02 2.01205621e-02 5.89624877e-03
 3.64166385e-03 2.46619985e-03 6.56004343e-04]
[83954.57885113 15426.76164579 12322.49063561  6670.6221544
  5242.37845034  4314.12352499  2225.00914236]


In [8]:
from nba_api.stats.endpoints import BoxScoreAdvancedV2

BoxScoreAdvancedV2.player_stats().get_data_frames()[0]

TypeError: 'NoneType' object is not callable

In [22]:
from nba_api.stats.endpoints import playerawards

playerawards.PlayerAwards(player_id=2544).get_data_frames()[0]

,PERSON_ID,FIRST_NAME,LAST_NAME,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3
0,2544,LeBron,James,Cleveland Cavaliers,All-Defensive Team,1,2008-09,None,None,1610612739,Award,Kia Motors,KIADT,None
1,2544,LeBron,James,Cleveland Cavaliers,All-Defensive Team,1,2009-10,None,None,1610612739,Award,Kia Motors,KIADT,None
2,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2010-11,None,None,1610612748,Award,Kia Motors,KIADT,None
3,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2011-12,None,None,1610612748,Award,Kia Motors,KIADT,None
4,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2012-13,None,None,1610612748,Award,Kia Motors,KIADT,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,2544,LeBron,James,Cleveland Cavaliers,NBA Sporting News Most Valuable Player of the ...,None,2005-06,None,None,None,Award,Sporting News,SNMVP,None
145,2544,LeBron,James,Cleveland Cavaliers,NBA Sporting News Rookie of the Year,None,2003-04,None,None,None,Award,Sporting News,SNROY,None
146,2544,LeBron,James,USA,Olympic Bronze Medal,None,2004,None,None,None,Award,Olympic,Bronze,None
147,2544,LeBron,James,USA,Olympic Gold Medal,None,2008,None,None,None,Award,Olympic,Gold,None


In [27]:
total_names = total_players[["PERSON_ID", "DISPLAY_FIRST_LAST"]]

# total_names1 = total_names[:int(len(total_names)/2)]
# with open("data_all_team_stats.pkl", "rb") as f:
#     data_all_team_stats = pickle.load(f)
#
# total_stats = pd.DataFrame()
# for index, row in tqdm(total_names1.iterrows()):
#     stats = playercareerstats.PlayerCareerStats(player_id=row["PERSON_ID"]).get_data_frames()[0]
#
#     stats = stats.drop(stats.columns[[2, 4, 7]], axis=1)
#
#     awards = playerawards.PlayerAwards(player_id=row["PERSON_ID"]).get_data_frames()[0]
#
#     awards = awards.drop(awards.columns[[0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13]], axis=1) #Tolgo anche PERSON_ID
#     awards = awards.rename(columns={"SEASON": "SEASON_ID"})
#     awards = awards.loc[awards["DESCRIPTION"]=="NBA Most Valuable Player"]
#
#     #merge
#     stats=stats.merge(awards, how="left", on="SEASON_ID")
#     #1 to MVP, 0 to other
#     stats["DESCRIPTION"] = stats["DESCRIPTION"].replace("NBA Most Valuable Player", 1) #Replace the description MVP with 1
#     stats["DESCRIPTION"] = stats["DESCRIPTION"].fillna(0) #Replace NULL with 0
#     stats = stats.rename(columns={"DESCRIPTION": "MVP"})
#
#     #merge della classifica in cui è arrivata la squadra in una certa stagione
#     stats = stats.merge(data_all_team_stats, how="inner", on=["SEASON_ID", "TEAM_ID"])
#
#     total_stats = pd.concat([total_stats, stats],axis=0,ignore_index=True)
#
#
# #Tolgo la formattazione "year1-year2" (2010-11) e lascio solo il secondo anno (2011)
# total_stats["SEASON_ID"] = [int("20"+x.split("-")[1]) for x in list(total_stats["SEASON_ID"])] #Problema se si prendono dati < anno 2000
# #Prendo i dati superiore all'anno FROM_YEAR
# total_stats= total_stats[total_stats["SEASON_ID"] > from_year]
#
# tot1 = total_stats
# with open("player_stats1.pkl",'wb') as f:
#     pickle.dump(total_stats,f)

total_stats = pd.DataFrame()
total_names2 = total_names[int((len(total_names)/2)+1):]

for index, row in tqdm(total_names2.iterrows()):
    stats = playercareerstats.PlayerCareerStats(player_id=row["PERSON_ID"]).get_data_frames()[0]

    stats = stats.drop(stats.columns[[2, 4, 7]], axis=1)

    awards = playerawards.PlayerAwards(player_id=row["PERSON_ID"]).get_data_frames()[0]

    awards = awards.drop(awards.columns[[0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13]], axis=1) #Tolgo anche PERSON_ID
    awards = awards.rename(columns={"SEASON": "SEASON_ID"})
    awards = awards.loc[awards["DESCRIPTION"]=="NBA Most Valuable Player"]

    #merge
    stats=stats.merge(awards, how="left", on="SEASON_ID")
    #1 to MVP, 0 to other
    stats["DESCRIPTION"] = stats["DESCRIPTION"].replace("NBA Most Valuable Player", 1) #Replace the description MVP with 1
    stats["DESCRIPTION"] = stats["DESCRIPTION"].fillna(0) #Replace NULL with 0
    stats = stats.rename(columns={"DESCRIPTION": "MVP"})

    #merge della classifica in cui è arrivata la squadra in una certa stagione
    stats = stats.merge(data_all_team_stats, how="inner", on=["SEASON_ID", "TEAM_ID"])

    total_stats = pd.concat([total_stats, stats],axis=0,ignore_index=True)


#Tolgo la formattazione "year1-year2" (2010-11) e lascio solo il secondo anno (2011)
total_stats["SEASON_ID"] = [int("20"+x.split("-")[1]) for x in list(total_stats["SEASON_ID"])] #Problema se si prendono dati < anno 2000
#Prendo i dati superiore all'anno FROM_YEAR
total_stats= total_stats[total_stats["SEASON_ID"] > from_year]

tot2 = total_stats
with open("player_stats2.pkl",'wb') as f:
    pickle.dump(total_stats,f)

#total_stats = stats.concat([tot1, tot2], axis=0, ignore_index=True)

973it [41:07,  2.54s/it]


In [28]:

with open("player_stats1.pkl",'rb') as f:
    tot1 = pickle.load(f)
with open("player_stats2.pkl",'rb') as f:
    tot2 = pickle.load(f)

total_stats = pd.concat([tot1, tot2], axis=0, ignore_index=False)

with open("player_stats.pkl", "wb") as f:
    pickle.dump(total_stats,f)

total_stats


,PLAYER_ID,SEASON_ID,TEAM_ID,PLAYER_AGE,GP,MIN,FGM,FGA,FG_PCT,FG3M,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,MVP,CONF_RANK
0,949,2097,1610612763,20.0,80,2804.0,550,1214,0.453,7,...,339,555,175,79,79,225,199,1494,0.0,14
1,949,2098,1610612763,21.0,82,2950.0,653,1347,0.485,21,...,354,581,213,89,76,257,201,1829,0.0,11
2,949,2099,1610612763,22.0,50,2021.0,386,893,0.432,11,...,260,374,172,69,55,186,137,1152,0.0,14
7,949,2004,1610612737,27.0,53,1955.0,383,790,0.485,5,...,354,495,127,44,19,131,147,1065,0.0,12
8,949,2004,1610612757,27.0,32,729.0,118,264,0.447,4,...,96,144,47,24,18,53,75,319,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5977,1627826,2019,1610612747,22.0,33,516.0,112,193,0.580,0,...,108,162,25,4,27,33,73,281,0.0,10
5978,1627826,2019,1610612746,22.0,26,524.0,100,186,0.538,0,...,139,200,38,10,24,37,64,244,0.0,8
5979,1627826,2020,1610612746,23.0,72,1326.0,236,385,0.613,0,...,346,543,82,16,66,61,168,596,0.0,2
5980,1627826,2021,1610612746,24.0,72,1609.0,257,394,0.652,1,...,330,519,90,24,62,81,187,650,0.0,4
